Créé le Samedi 06 février 2021  

**Groupe 3 - Evolution des arnaques en ligne dans le temps**

**Scraper pour ZDNnet**

auteur: Marianne Manson

In [1]:
#Lien de test
test_link = "https://www.zdnet.fr/actualites/phishing-les-clients-de-la-poste-victimes-d-arnaques-au-colis-39913457.htm"

In [2]:
#import de lib
import pandas as pd
import numpy as np
import re
from datetime import datetime
from requests import get
from bs4 import BeautifulSoup

In [3]:
# id_article titre  date_publication  date_extraction  content_brut  content_art  type  resume  mots_clef  auteurs  url_art 
df_scrap = pd.DataFrame(columns=['id_article','titre','date_publication','date_extraction','content_brut','content_art','type','resume','mots_clef','auteurs','url_art','id_source'])
df_scrap

,id_article,titre,date_publication,date_extraction,content_brut,content_art,type,resume,mots_clef,auteurs,url_art,id_source


In [4]:
req = get(test_link)
html_soup = BeautifulSoup(req.text, 'html.parser')
html_soup

<!DOCTYPE html>

<!-- BEGIN html -->
<!--[if lt IE 7 ]><html class="no-js ie ie6" lang="fr" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:v="http://rdf.data-vocabulary.org/#"><![endif]-->
<!--[if IE 7 ]><html class="no-js ie ie7" lang="fr" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:v="http://rdf.data-vocabulary.org/#"><![endif]-->
<!--[if IE 8 ]><html class="no-js ie ie8" lang="fr" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:v="http://rdf.data-vocabulary.org/#"><![endif]-->
<!--[if (gte IE 9)|!(IE)]><!-->
<html class="no-js" lang="fr" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:v="http://rdf.data-vocabulary.org/#">
<!--<![endif]-->
<!-- BEGIN head-->
<head>
<script type="text/javascript">
    window.gdprAppliesGlobally=true;(function(){function a(e){if(!window.frames[e]){if(document.body&&document.body.firstChild){var t=document.body;var n=document.createElement("iframe");n.style.display="none";n.name=e;n.title=e;t.insertBefore(n,t.firstChild)}
    else{setTi

In [6]:
def scraping(url):
    '''Documentation
    
    Parameters:
        url: url de la page de l'article
    
    Out:
        new_row: données sous format dictionnaire
    '''
    req = get(url)
    html_soup = BeautifulSoup(req.text, 'html.parser')
    #Données vides
    no_data = np.nan
    #id_article
    id_art = no_data
    # titre
    zone_titre = html_soup.find("h1")
    if zone_titre is not None:
        titre = zone_titre.text
    else:
        titre = no_data
    # auteurs, date_publication
    zone_infos = html_soup.find("div",{'class':"byline"})
    print(zone_infos)
    zone_infos = zone_infos.find("p",{'class':"meta"})
    zone_auteur = zone_infos.find("span")
    zone_auteur = zone_auteur.find("span")
    if zone_auteur is not None:
        auteur = zone_auteur.text
    else:
        auteur = no_data
    zone_date = zone_infos.find("time")
    if zone_date is not None:
        date_pub = zone_date['datetime']
        format_end = date_pub[-5:]
        date_pub = datetime.strptime(date_pub,'%Y-%m-%dT%H:%M:%S+'+format_end).date()
    #date_extraction
    date_extr = datetime.today().date()
    # content_brut
    content_html = html_soup.find("div",{'class':"storyBody"}) # Ou on doit prendre la page entière ?
    # content_art
    if content_html is not None:
         content = content_html.text
    else:
        content = no_data
    # resume
    resume_zone = html_soup.find("p",{'class':"summary"})
    if resume_zone is not None:
        resume = resume_zone.text
    else:
        resume = no_data
    #tags
    zone_tags = html_soup.find("p",{'class':"relatedTopics"})
    liste_lien_tags = zone_tags.find_all("a")
    if liste_lien_tags is not None:
        liste_tags = []
        for lien in liste_lien_tags:
            liste_tags.append(lien.text)
    # dictionnaire
    new_row = {'id_article': id_art,
               'titre': titre,
               'date_publication': date_pub,
               'date_extraction' : date_extr,
               'content_brut' : content_html,
               'content_art' : content,
               'type' : 'html',
               'resume' : resume,
               'mots_clef' : liste_tags,
               'auteurs' : auteur,
               'url_art' : url,
               'id_source' : 'zdnet'
              }
    return new_row

In [14]:
req = get(test_link)
html_soup = BeautifulSoup(req.text, 'html.parser')
html_soup.find("p",{'class':"summary"})

<p class="summary"><span>Sécurité :</span> De faux messages de phishing visant les clients de La Poste circulent, usurpant l’identité du service pour inciter les victimes à communiquer leurs données personnelles ou à payer une petite somme pour récupérer un colis inexistant. Une campagne qui dure depuis plusieurs mois.</p>

In [ ]:
date_pub=html_soup.find("div",{'class':"byline"}).find("p",{'class':"meta"}).find("time")['datetime']
format_end = date_pub[-5:]
datetime.strptime(date_pub,'%Y-%m-%dT%H:%M:%S+'+format_end).date()

datetime.date(2020, 11, 22)

In [ ]:
date_pub

'2020-11-22T08:00:00+02:00'

In [ ]:
new_row = scraping(test_link)
df_test = df_scrap.append(new_row, ignore_index=True)
df_test

,id_article,titre,date_publication,date_extraction,content_brut,content_art,type,resume,mots_clef,auteurs,url_art,id_source
0,NaN,\n Phishing : Les clients de La Poste v...,2020-11-22,2021-02-06,"[\n, [\n, [\n, <img alt=""Phishing : Les client...",\n\n\n\n\n\nA l’approche des fêtes et en plein...,html,Sécurité : De faux messages de phishing visant...,"[Cybersécurité, Cybercriminalité, Phishing]",Louis Adam,https://www.zdnet.fr/actualites/phishing-les-c...,zdnet
